# QESN Complete Classification Demo (Adaptive 90-95% Accuracy Pipeline)

This notebook rebuilds the end-to-end QESN-MABe classification workflow using the 
exported checkpoint (≈92.6% accuracy) and the adaptive optimisation plan described in
`Mejorar_Precision_Modelo_Cuantico.md`.\n\nIt is designed to run against the new exposure dataset and report production-grade metrics.

## Objetivos

- Cargar el modelo exportado (`model_weights.bin`, `model_config.json`) y validar paridad.
- Aplicar limpieza y balanceo temporal sobre el dataset de exposición.
- Introducir ajustes físicos adaptativos (dt y energía) antes de cada ventana.
- Evaluar accuracy, macro-F1 y matriz de confusión para las 37 clases.
- Generar reportes listos para las demos profesionales.

## 1. Importaciones y Configuración

In [ ]:
import os
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, Iterable, List, Optional, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

from python.model_loader import load_inference
from python.qesn_inference import QESNInference  # compat for type hints

plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['figure.dpi'] = 140
plt.rcParams['font.size'] = 10
sns.set_theme(style='whitegrid', palette='husl')

## 2. Rutas y Carga del Modelo

Ajusta las variables de entorno `QESN_MODEL_DIR` y `QESN_DATASET_ROOT` para personalizar los paths.

In [ ]:
MODEL_DIR = os.getenv('QESN_MODEL_DIR')
DATASET_ROOT = Path(os.getenv('QESN_DATASET_ROOT', 'data/exposure_dataset'))
TRACKING_DIR = DATASET_ROOT / 'tracking'
LABELS_CSV = DATASET_ROOT / 'labels.csv'

print('Model directory   :', MODEL_DIR or '(using default kaggle_model/)')
print('Dataset root      :', DATASET_ROOT)
print('Tracking parquet  :', TRACKING_DIR)
print('Labels CSV        :', LABELS_CSV)

# Cargar inferencia base
base_model = load_inference(MODEL_DIR)
print('
Modelo cargado:')
print('  grid      :', base_model.grid_width, 'x', base_model.grid_height)
print('  window    :', base_model.window_size)
print('  stride    :', base_model.stride)
print('  clases    :', len(base_model.class_names))

## 3. Utilidades de Carga y Limpieza de Datos

Las funciones siguientes asumen el formato estándar MABe: un `labels.csv` con columnas `video_id`, `frame`, `behavior` y un directorio de `parquet` con keypoints (`frame`, `track_id`, `keypoint`, `x`, `y`, `confidence`). Adapta las funciones si tu dataset tiene cambios de esquema.

In [ ]:
MABE_BEHAVIORS = base_model.class_names
NUM_BEHAVIORS = len(MABE_BEHAVIORS)
KEYPOINTS_PER_MOUSE = 18
MICE_PER_FRAME = 4

@dataclass
class WindowSample:
    video_id: str
    start_frame: int
    end_frame: int
    keypoints: np.ndarray  # shape: (frames, mice, keypoints, 3)
    true_label: int

def load_labels(labels_csv: Path) -> pd.DataFrame:
    if not labels_csv.exists():
        raise FileNotFoundError(f'Archivo de etiquetas no encontrado: {labels_csv}')
    df = pd.read_csv(labels_csv)
    required = {'video_id', 'frame', 'behavior'}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f'Faltan columnas en labels.csv: {missing}')
    df['behavior'] = df['behavior'].astype(str)
    df = df[df['behavior'].isin(MABE_BEHAVIORS)].copy()
    return df

def load_tracking(video_id: str) -> pd.DataFrame:
    parquet_path = TRACKING_DIR / f'{video_id}.parquet'
    if not parquet_path.exists():
        raise FileNotFoundError(f'No existe el parquet de tracking: {parquet_path}')
    return pd.read_parquet(parquet_path)

def reshape_keypoints(df: pd.DataFrame) -> np.ndarray:
    required = {'frame', 'track_id', 'keypoint', 'x', 'y', 'confidence'}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f'Faltan columnas en tracking: {missing}')
    frames = int(df['frame'].max()) + 1
    kp_array = np.full((frames, MICE_PER_FRAME, KEYPOINTS_PER_MOUSE, 3), np.nan, dtype=np.float32)
    grouped = df.groupby(['frame', 'track_id', 'keypoint'])
    for (frame, track, kp), row in grouped:
        if track >= MICE_PER_FRAME or kp >= KEYPOINTS_PER_MOUSE:
            continue
        kp_array[frame, track, kp, 0] = row['x']
        kp_array[frame, track, kp, 1] = row['y']
        kp_array[frame, track, kp, 2] = row['confidence']
    return kp_array

def interpolate_missing(keypoints: np.ndarray) -> np.ndarray:
    mask = np.isnan(keypoints)
    if not mask.any():
        return keypoints
    filled = np.copy(keypoints)
    for axis in range(3):
        data = filled[..., axis]
        missing = np.isnan(data)
        if missing.any():
            idx = np.arange(data.shape[0])
            for mouse in range(data.shape[1]):
                for kp in range(data.shape[2]):
                    series = data[:, mouse, kp]
                    miss = np.isnan(series)
                    if miss.all():
                        continue
                    valid_idx = idx[~miss]
                    valid_vals = series[~miss]
                    series[miss] = np.interp(idx[miss], valid_idx, valid_vals)
                    data[:, mouse, kp] = series
    filled[..., 2] = np.clip(filled[..., 2], 0.0, 1.0)
    return filled

def compute_window_label(labels_df: pd.DataFrame, video_id: str, start: int, end: int) -> Optional[int]:
    window_labels = labels_df[(labels_df['video_id'] == video_id) & (labels_df['frame'] >= start) & (labels_df['frame'] < end)]
    if window_labels.empty:
        return None
    counts = window_labels['behavior'].value_counts()
    return MABE_BEHAVIORS.index(counts.idxmax())

def iter_windows(keypoints: np.ndarray, window: int, stride: int) -> Iterable[Tuple[int, int, np.ndarray]]:
    total_frames = keypoints.shape[0]
    for start in range(0, total_frames - window + 1, stride):
        end = start + window
        yield start, end, keypoints[start:end]

def generate_samples(video_id: str, keypoints: np.ndarray, labels_df: pd.DataFrame, window: int, stride: int) -> List[WindowSample]:
    samples: List[WindowSample] = []
    for start, end, chunk in iter_windows(keypoints, window, stride):
        label = compute_window_label(labels_df, video_id, start, end)
        if label is None:
            continue
        samples.append(WindowSample(video_id, start, end, chunk, label))
    return samples

## 4. Modelo Adaptativo

Implementamos un contenedor que ajusta `dt`, `energy_injection` y `coupling_strength` según la cinemática de cada ventana antes de delegar en `QESNInference`.

In [ ]:
class AdaptiveQESN:
    def __init__(self, model: QESNInference):
        self.model = model

    @staticmethod
    def _window_stats(keypoints: np.ndarray) -> Dict[str, float]:
        displacements = np.diff(keypoints[..., :2], axis=0)
        speeds = np.linalg.norm(displacements, axis=-1)
        mean_speed = np.nanmean(speeds) if speeds.size else 0.0
        max_speed = np.nanmax(speeds) if speeds.size else 0.0
        valid_ratio = np.mean(keypoints[..., 2] > 0.5)
        return {
            'mean_speed': float(np.nan_to_num(mean_speed)),
            'max_speed': float(np.nan_to_num(max_speed)),
            'valid_ratio': float(np.nan_to_num(valid_ratio))
        }

    def _configure_physics(self, stats: Dict[str, float]):
        base_coupling = 0.5
        coupling = np.clip(base_coupling + 0.05 * (stats['mean_speed'] / 15.0 - 0.5), 0.45, 0.52)
        diffusion = np.clip(coupling, 0.45, 0.52)
        dt = 0.0015 if stats['max_speed'] > 25.0 else 0.0020
        energy = 0.05 * np.clip(0.8 + stats['valid_ratio'], 0.9, 1.1)
        if stats['valid_ratio'] < 0.6:
            energy *= 1.1
        self.model.foam.setCouplingStrength(coupling)
        self.model.foam.setDiffusionRate(diffusion)
        self.model.foam.setDecayRate(0.001)
        self.model.dt = dt
        self.model.energy_injection = energy

    def predict_window(self, window: WindowSample, video_width: int, video_height: int):
        stats = self._window_stats(window.keypoints)
        self._configure_physics(stats)
        pred_idx, probs, pred_name = self.model.predict(window.keypoints, video_width, video_height, window_size=self.model.window_size)
        return {
            'video_id': window.video_id,
            'start': window.start_frame,
            'end': window.end_frame,
            'pred_idx': int(pred_idx),
            'pred_name': pred_name,
            'probabilities': probs,
            'stats': stats
        }

adaptive_model = AdaptiveQESN(base_model)

## 5. Evaluación del Dataset

La función `evaluate_dataset` recorre todos los videos, genera ventanas de 60 frames (stride 30) y calcula accuracy/macro-F1. Ajusta `max_videos` para reducir tiempo durante depuración.

In [ ]:
def evaluate_dataset(labels_df: pd.DataFrame, max_videos: Optional[int] = None) -> Dict[str, object]:
    if not TRACKING_DIR.exists():
        raise FileNotFoundError(f'Directorio de tracking no disponible: {TRACKING_DIR}')
    video_ids = sorted(labels_df['video_id'].unique())
    if max_videos:
        video_ids = video_ids[:max_videos]

    y_true: List[int] = []
    y_pred: List[int] = []
    records: List[Dict[str, object]] = []

    for video_id in tqdm(video_ids, desc='Evaluating videos'):
        tracking_df = load_tracking(video_id)
        keypoints = reshape_keypoints(tracking_df)
        keypoints = interpolate_missing(keypoints)
        samples = generate_samples(video_id, keypoints, labels_df, base_model.window_size, base_model.stride)
        if not samples:
            continue
        video_width = int(np.nanmax(keypoints[..., 0]))
        video_height = int(np.nanmax(keypoints[..., 1]))
        for sample in samples:
            result = adaptive_model.predict_window(sample, video_width, video_height)
            y_true.append(sample.true_label)
            y_pred.append(result['pred_idx'])
            records.append({
                'video_id': video_id,
                'start_frame': sample.start_frame,
                'end_frame': sample.end_frame,
                'true_label': sample.true_label,
                'pred_label': result['pred_idx'],
                'true_behavior': MABE_BEHAVIORS[sample.true_label],
                'pred_behavior': result['pred_name'],
                'confidence': float(result['probabilities'][result['pred_idx']]),
                'mean_speed': result['stats']['mean_speed'],
                'max_speed': result['stats']['max_speed'],
                'valid_ratio': result['stats']['valid_ratio']
            })

    if not y_true:
        raise RuntimeError('No se generaron ventanas con etiquetas válidas.')

    accuracy = accuracy_score(y_true, y_pred)
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    report = classification_report(y_true, y_pred, target_names=MABE_BEHAVIORS, digits=3)
    cm = confusion_matrix(y_true, y_pred, labels=list(range(NUM_BEHAVIORS)))
    results_df = pd.DataFrame.from_records(records)

    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'report': report,
        'confusion_matrix': cm,
        'predictions': results_df
    }

## 6. Ejecutar Evaluación

Si aún no tienes el dataset preparado, ejecuta primero el pipeline de extracción. Ajusta `max_videos` para pruebas rápidas.

In [ ]:
labels_df = load_labels(LABELS_CSV)
print(f'Videos disponibles: {labels_df.video_id.nunique()}')
print(labels_df.head())

# Cambia max_videos a None para procesar todo el dataset
EVAL_RESULTS = evaluate_dataset(labels_df, max_videos=None)
print(f"Accuracy: {EVAL_RESULTS['accuracy']*100:.2f}%")
print(f"Macro F1: {EVAL_RESULTS['macro_f1']*100:.2f}%")

In [ ]:
low_conf_df = EVAL_RESULTS['predictions'][EVAL_RESULTS['predictions'].confidence < 0.10]
print(f'Bajas confianzas detectadas: {len(low_conf_df)} ventanas')
display(low_conf_df[['video_id', 'start_frame', 'end_frame', 'true_behavior', 'pred_behavior', 'confidence', 'valid_ratio', 'mean_speed']].head())


## 7. Visualización de Resultados

In [ ]:
pred_df = EVAL_RESULTS['predictions']
cm = EVAL_RESULTS['confusion_matrix']

fig, ax = plt.subplots(figsize=(14, 12))
sns.heatmap(cm, ax=ax, cmap='viridis', linewidths=0.1, linecolor='gray', cbar=True, square=True)
ax.set_title('Confusion Matrix - QESN Adaptive Inference')
ax.set_xlabel('Predicted')
ax.set_ylabel('True')
ax.set_xticks(np.arange(NUM_BEHAVIORS) + 0.5)
ax.set_xticklabels(MABE_BEHAVIORS, rotation=90)
ax.set_yticks(np.arange(NUM_BEHAVIORS) + 0.5)
ax.set_yticklabels(MABE_BEHAVIORS, rotation=0)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
sns.histplot(pred_df['confidence'], bins=30, kde=True)
plt.title('Distribution of Prediction Confidence')
plt.xlabel('confidence')
plt.ylabel('count')
plt.tight_layout()
plt.show()

print('Classification Report:
')
print(EVAL_RESULTS['report'])

## 8. Persistencia y Reporting

Guarda los resultados para integrarlos con los demos multimedia.

In [ ]:
OUTPUT_DIR = DATASET_ROOT / 'results'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

pred_path = OUTPUT_DIR / 'qesn_adaptive_predictions.parquet'
report_path = OUTPUT_DIR / 'qesn_classification_report.txt'
cm_path = OUTPUT_DIR / 'qesn_confusion_matrix.npy'

pred_df.to_parquet(pred_path, index=False)
np.save(cm_path, EVAL_RESULTS['confusion_matrix'])
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(f"Accuracy: {EVAL_RESULTS['accuracy']*100:.3f}%\n")
    f.write(f"Macro F1: {EVAL_RESULTS['macro_f1']*100:.3f}%\n\n")
    f.write(EVAL_RESULTS['report'])

print('Resultados guardados en:', OUTPUT_DIR)

## 9. Próximos Pasos

- Repetir la evaluación con validación cruzada (3 folds) si se requiere certificación.
- Ajustar los parámetros adaptativos (`dt`, energía, coupling) según el análisis de los resultados guardados en `results/`.
- Integrar estos artefactos en los demos (`demo_espectacular.py`, notebooks profesionales).